In [1]:
import torch

In [2]:
tensor_path = "/home/julian/dev/math-ml/experiments/llama2_test_cot/output_2025-06-11_11-50.pt"
exp_results = torch.load(tensor_path)

In [3]:
exp_results.keys()

dict_keys(['prompt0', 'prompt1', 'prompt2', 'prompt3', 'prompt4', 'prompt5', 'prompt6', 'prompt7', 'prompt8', 'prompt9', 'prompt10', 'prompt11', 'prompt12', 'prompt13', 'prompt14'])

In [5]:
exp_results['prompt1'].keys()
#exp_results['prompt1']['top_p_tokens']

dict_keys(['generated_tokens', 'decoded_tokens', 'top_p_tokens', 'top_p_logits', 'top_p_probs', 'prompt', 'question', 'ground_truth', 'latency'])

In [6]:
prompt = 'prompt5'
print(f"Answer: {' '.join(exp_results[prompt]['decoded_tokens'])}")
print()
print(exp_results[prompt]['ground_truth'])

Answer: A : 
 K yl ar wants to buy  1 6 glass es . The price for  1 6 glass es should be  5 *  1 6 =  8 0 . The price for every second glass is  6 0 % of the first one . So the price for a  6 0 % of  5 dollars ( which is  3 dollars ). So we just need to multiply the  6 0 % of the price for one glass and the  1 6 glass es . 
 6 0 % *  5 *  1 6 =  6 0 % *  8 0 
 6 0 % *  4 0 =  2 4 dollars . 
 Now , he has to pay for the  1 6 glass es and the  2 4 dollars of tax es . 
 The answer would be  8 0 +  2 4 =  1 0 4 dollars . 
 Q

The discount price of one glass is 60/100 * 5 = $<<60/100*5=3>>3.
If every second glass is cheaper, that means Kylar is going to buy 16 / 2 = <<16/2=8>>8 cheaper glasses.
So for the cheaper glasses, Kylar is going to pay 8 * 3 = $<<8*3=24>>24.
And for the regular-priced glasses, Kylar will pay 8 * 5 = $<<8*5=40>>40.
So in total Kylar needs to pay 24 + 40 = $<<24+40=64>>64 for the glasses he wants to buy.
#### 64


In [ ]:
exp_results['prompt1']['top_p_probs']

tensor([0.4160, 0.2227, 0.1270, 0.0771, 0.0530, 0.0498, 0.0118, 0.0087, 0.0036,
        0.0032, 0.0030, 0.0030, 0.0014, 0.0012, 0.0010, 0.0010, 0.0007, 0.0006,
        0.0006, 0.0006, 0.0006], dtype=torch.bfloat16)

In [45]:
exp_results['prompt0']['top_p_tokens'][9]

tensor([53308,  2055,   576])

In [46]:
exp_results['prompt0']['generated_tokens'][9]

tensor(53308)

In [40]:
len(exp_results['prompt0']['decoded_tokens'])

10

In [30]:
raw_test = 'The answer is 72.0'
test = 72.0

In [28]:
reverse = test[::-1]
print(reverse)

cba


In [34]:
test = str(test)
raw_test_rvs = raw_test[::-1]
for char in test:
    for idx,char_b in enumerate(raw_test_rvs):
        if char == char_b:
            indice = len(raw_test_rvs) - idx -1
            print(indice)
            break

14
15
16
17
